In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np

In [ ]:
df = pd.read_csv('Mental_Health_FAQ.csv')
df = df.drop('Question_ID', axis=1)
df.head()

,Questions,Answers
0,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [ ]:
questions = [q for q in df['Questions'].values]
answers = [a for a in df['Answers'].values]

In [ ]:
question = "Are there any differences between depression and sadness?"
questions.append(question)

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
lemmatized_q = []

for q in questions:
  doc = nlp(q)
  lemmatized_q.append(" ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct]))

In [ ]:
lemmatized_q[:5]

['mean mental illness',
 'mental illness affect',
 'cause mental illness',
 'warning sign mental illness',
 'people mental illness recover']

In [ ]:
cv = CountVectorizer()
cv_q = cv.fit_transform(lemmatized_q)

In [ ]:
tf = TfidfVectorizer()
tf_idf_q = tf.fit_transform(lemmatized_q)

In [ ]:
cv_df = pd.DataFrame(cv_q.toarray(), columns=cv.get_feature_names_out())
cv_df.head()

,19,25,addiction,adhd,adult,advance,affect,age,alcohol,allow,...,unwell,use,vaping,warning,well,will,work,worried,young,youth
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
tf_df = pd.DataFrame(tf_idf_q.toarray(), columns=tf.get_feature_names_out())
tf_df.head()

,19,25,addiction,adhd,adult,advance,affect,age,alcohol,allow,...,unwell,use,vaping,warning,well,will,work,worried,young,youth
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.750128,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.600065,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
q_cv = cv_df.to_numpy()
q_tf = tf_df.to_numpy()

In [ ]:
from scipy.spatial.distance import cosine, jaccard
from scipy.stats import pearsonr
from sklearn.metrics.cluster import rand_score

COS

In [ ]:
best_cv_cos = 0
ind = -1

for i in range(len(questions) - 1):
  curr = 1 - cosine(q_cv[i, :], q_cv[-1, :])
  if curr > best_cv_cos:
    best_cv_cos = curr
    ind = i

print("cos_cv:", question)
print(answers[ind])

cos_cv: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel worthless or

In [ ]:
best_tf_cos = 0
ind = -1

for i in range(len(questions) - 1):
  curr = 1 - cosine(q_tf[i, :], q_tf[-1, :])
  if curr > best_tf_cos:
    best_tf_cos = curr
    ind = i

print("cos_tf_idf:", question)
print(answers[ind])

cos_tf_idf: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel worthles

Pearson

In [ ]:
best_cv_p = 0
ind = -1

for i in range(len(questions) - 1):
  curr = pearsonr(q_cv[i, :], q_cv[-1, :])[0]
  if curr > best_cv_p:
    best_cv_p = curr
    ind = i

print("pearson_cv:", question)
print(answers[ind])

pearson_cv: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel worthles

In [ ]:
best_tf_p = 0
ind = -1

for i in range(len(questions) - 1):
  curr = pearsonr(q_tf[i, :], q_tf[-1, :])[0]
  if curr > best_tf_p:
    best_tf_p = curr
    ind = i

print("pearson_tf_idf:", question)
print(answers[ind])

pearson_tf_idf: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel wort

Jaccard

In [ ]:
best_cv_j = 0
ind = -1

for i in range(len(questions) - 1):
  curr = 1 - jaccard(q_cv[i, :], q_cv[-1, :])
  if curr > best_cv_j:
    best_cv_j = curr
    ind = i

print("jaccard_cv:", question)
print(answers[ind])

jaccard_cv: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel worthles

In [ ]:
best_tf_j = 0
ind = -1

for i in range(len(questions) - 1):
  curr = 1 - jaccard(q_tf[i, :], q_tf[-1, :])
  if curr > best_tf_j:
    best_tf_j = curr
    ind = i

print("jaccard_tf_idf:", question)
print(answers[ind])

jaccard_tf_idf: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel wort

Rand_score

In [ ]:
best_cv_r = 0
ind = -1

for i in range(len(questions) - 1):
  curr = rand_score(q_cv[i, :], q_cv[-1, :])
  if curr > best_cv_r:
    best_cv_r = curr
    ind = i

print("rand_score_cv:", question)
print(answers[ind])

rand_score_cv: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel worth

In [ ]:
best_tf_r = 0
ind = -1

for i in range(len(questions) - 1):
  curr = rand_score(q_tf[i, :], q_tf[-1, :])
  if curr > best_tf_r:
    best_tf_r = curr
    ind = i

print("rand_score_tf_idf:", question)
print(answers[ind])

rand_score_tf_idf: Are there any differences between depression and sadness?
Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. 
 Other ways to talk about sadness might be ‘feeling low,’ ‘feeling down,’ or ‘feeling blue.’ A person may say they are feeling ‘depressed,’ but if it goes away on its own and doesn’t impact life in a big way, it probably isn’t the illness of depression. 
 Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It’s much more than sadness or low mood. People who experience depression may feel w